In [1]:
import os, sys, re, datetime, random, gzip, json
from tqdm.autonotebook import tqdm
import pandas as pd
import numpy as np
import glob
from pathlib import Path
from itertools import accumulate
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import BertTokenizer, BertModel, BertForSequenceClassification

from time import time
from math import ceil
from multiprocessing import Pool
from sentence_transformers import SentenceTransformer, models, losses, InputExample

import pytorch_lightning as pl
from pytorch_lightning.trainer.trainer import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.utilities.seed import seed_everything

from ray.tune.integration.pytorch_lightning import TuneReportCallback

PROJ_PATH = Path(os.path.join(re.sub("/BERT_ABSA.*$", '', os.getcwd()), 'BERT_ABSA'))
print(f'PROJ_PATH={PROJ_PATH}')
sys.path.insert(1, str(PROJ_PATH))
sys.path.insert(1, str(PROJ_PATH/'src'))
import utils

/tmp/ipykernel_5005/3000755598.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


PROJ_PATH=/home/hoang/github/BERT_ABSA


In [2]:
# glob.glob('../model/restaurants/*.ckpt')

## Parser

### XML parser

In [3]:
import os
import json
import pandas as pd
import re
import xml.etree.ElementTree as ET
from pathlib import Path

In [4]:
def parseXML(data_path):
    tree = ET.ElementTree(file=data_path)
    objs = list()
    for sentence in tree.getroot():
        obj = dict()
        obj['id'] = sentence.attrib['id']
        for item in sentence:
            if item.tag == 'text':
                obj['text'] = item.text
            elif item.tag == 'aspectTerms':
                obj['aspects'] = list()
                for aspectTerm in item:
                    if aspectTerm.attrib['polarity'] != 'conflict':
                        obj['aspects'].append(aspectTerm.attrib)
            elif item.tag == 'aspectCategories':
                obj['category'] = list()
                for category in item:
                    obj['category'].append(category.attrib)
        if 'aspects' in obj and len(obj['aspects']):
            objs.append(obj)
    return objs

def convert_to_dataframe(objs):
    output = []
    for sentence in objs:
        id = sentence['id']
        text = sentence['text']
        aspects = sentence['aspects']
        for aspect in aspects:
            term = aspect['term']
            label = aspect['polarity']
            output.append([id, text, term, label])
    output = sorted(output, key=lambda x: x[0])
    df = pd.DataFrame(output, columns=['id', 'text', 'term', 'label'])
    return df

### Dependency Tree parser

In [5]:
from stanfordcorenlp import StanfordCoreNLP
import json
from nltk.parse.corenlp import CoreNLPDependencyParser
# nlps = StanfordCoreNLP(str(PROJ_PATH / 'misc/stanford-corenlp-4.3.2'))
to_strip_chars = ".,\(\)"
BERT_MODEL = 'bert-base-uncased'

In [6]:
def process_bert_tokens(tokens, DEBUG=False):
    
    to_strip_chars = ".,\(\)"
    token_check_list_1 = ["s", "re", "m", "ve", "ll", "d"]
    
    tokens += [""]
    
    current_token_group = []
    output = []
    last_token = None
    for token_idx, token in enumerate(tokens):
        
        next_token = None
        if token_idx + 1 < len(tokens):
            next_token = tokens[token_idx + 1]

        reset = True
        if token == "'":
            reset = False
        elif token.startswith("##"):
            reset = False
        elif last_token is not None:
            if token in token_check_list_1:
                reset = False

        keep = True
        if (token == '.') and (next_token is None):
            keep = True
        elif (token == '.') and (next_token is not None) and (next_token != '.'):
            keep = True
        else:
            token = token.strip(to_strip_chars)
            if token == "":
                keep = False

        if reset:
            if len(current_token_group) > 0:
                output += [current_token_group]
            current_token_group = []
        if keep:
            if DEBUG:
                current_token_group += [token]
            else:
                current_token_group += [token_idx]
            last_token = token
            
    return output

def process_core_nlp_tokens(tokens, DEBUG=False):
    
    to_strip_chars = ".,\(\)"
    
    output = []
    for token_idx, token in enumerate(tokens):
        
        point_stripped_token = token.strip(".")
        if point_stripped_token == "":
            if DEBUG:
                output += ["."]
            else: 
                output += [token_idx]
        elif token.startswith("'"):
            if DEBUG:
                output[-1] += token
        else:
            isTokenEndsWithPoint = token.endswith(".")
            
            token = token.strip(to_strip_chars)
            if token != "":
                if DEBUG:
                    output += [token]
                    if isTokenEndsWithPoint:
                        output += [""]
                else: 
                    output += [token_idx]
                    if isTokenEndsWithPoint:
                        output += [-1]

    return output

In [7]:
# def tokenize_and_depparse(text):
#     text+=' '
#     text = re.sub(r'\. ',' . ',text).strip()
#     text = re.sub(r' {2,}',' ',text)
#     nlp_properties = {
#         'annotators': 'depparse',
# #         'tokenize.options': 'splitHyphenated=false,normalizeParentheses=false',
#         'tokenize.whitespace': True,  # all tokens have been tokenized before
#         'ssplit.isOneSentence': False,
#         'outputFormat': 'json',
#     }
    
#     try:
#         parsed = json.loads(nlps.annotate(text.strip(), nlp_properties))
#     except:
#         print('Error')
        
#     parsed = parsed['sentences']
#     tokens = []
#     tokens_dict = {}
#     tuples = []
#     tmplen = 0
#     for item in parsed:
#         for ite in item['tokens']:
#             tokens.extend([ite['word']])
#             tokens_dict[ite['index']] = ite['word']
# #         tokens.extend([ite['word'] for ite in item['tokens']])
#         tuples.extend([
#             (
#                 ite['dep'],
#                 ite['governor']-1+tmplen,
#                 ite['dependent']-1+tmplen
#             ) for ite in item['basicDependencies'] if ite['dep']!='ROOT'
#         ])
#         tmplen=len(tokens)
        
#     return tokens, tuples

def tokenize_and_depparse(text):
    '''
    # to_conll(10) will return the result in a format as follows:
    # id word lemma ctag tag feats head(head's id) rel(syntactic relation)
    # return values that is unknown will be shown as '_'
    # tag and ctag are considered to be equal
    '''
    parser = CoreNLPDependencyParser(url='http://localhost:9000')
    dep_parsed_sentence = parser.raw_parse(text)
    deps = dep_parsed_sentence.__next__()
    
    lines = deps.to_conll(10).split('\n')
    tokens = []
    tuples = []
    for line in lines:
        if line != '':
            result = line.split('\t')
            # id word lemma ctag tag feats head(head's id) rel(syntactic relation)
            tokens.append(result[1])
            if result[7] != 'ROOT':
                tuples.append((result[7], int(result[6])-1 , int(result[0])-1))   
    return tokens, tuples

def build_bert_token_whole(bert_token):
    result = ""
    for bert_sub_token in bert_token:
        if bert_sub_token.startswith("##"):
            result += bert_sub_token[2:]
        else:
            result += bert_sub_token
    return result

def map_corenlp_to_bert_from_indexes(corenlp_processed_indexes, bert_processed_indexes):
    output = {}
    for (corenlp_processed_index, bert_processed_index) in zip(corenlp_processed_indexes, bert_processed_indexes):
        output[corenlp_processed_index] = bert_processed_index
    return output

def map_corenlp_to_bert_from_indexes_2(corenlp_tokens, bert_tokens, corenlp_processed_indexes, bert_processed_indexes):

    output = {}
    
    bert_run_idx_global = 0
    for corenlp_idx in corenlp_processed_indexes:
        for bert_run_idx, bert_idx_group in enumerate(bert_processed_indexes[bert_run_idx_global:]):
        
            corenlp_token = corenlp_tokens[corenlp_idx]

            bert_token_group = map(lambda bert_idx: bert_tokens[bert_idx], bert_idx_group)
            bert_token = build_bert_token_whole(bert_token_group)

            lower_corenlp_token = corenlp_token.lower()
            lower_bert_token = bert_token.lower()
            if lower_corenlp_token.startswith(lower_bert_token) or lower_corenlp_token.endswith(lower_bert_token) or lower_bert_token.startswith(lower_corenlp_token) or lower_bert_token.endswith(lower_corenlp_token):
                bert_run_idx_global = bert_run_idx + 1
                output[corenlp_idx] = bert_idx_group
                break;
        
    return output
        
def map_corenlp_to_bert(corenlp_tokens, bert_tokens, DEBUG=False):
    corenlp_processed_indexes = process_core_nlp_tokens(corenlp_tokens, DEBUG)
    bert_processed_indexes = process_bert_tokens(bert_tokens, DEBUG)
#     return map_corenlp_to_bert_from_indexes(corenlp_processed_indexes, bert_processed_indexes)
    return map_corenlp_to_bert_from_indexes_2(corenlp_tokens, bert_tokens, corenlp_processed_indexes, bert_processed_indexes)

In [8]:
def clean_text(text):
    cleaned_text = text.lower().strip()
    return cleaned_text

def build_dep_parse_tree(text, verbose=False):
    '''
    Parse dependency tree and map CoreNLP index to BERT index
    
    Returns
    -------
        output_bert_v1s: list of source node indexes
        output_bert_v2s: list of target node indexes
        types: list of dependency relation
        
    Usage:
    ----------
        build_dep_parse_tree("I'm waiting ... It's 9am now.", True)
    '''
    check_existed_dict = {}
    
    text = clean_text(text)
    bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    bert_tokens = bert_tokenizer.tokenize(text)
    (corenlp_tokens, parse_tree_corenlp) = tokenize_and_depparse(text)
    
    corenlp_to_bert_map = map_corenlp_to_bert(corenlp_tokens, bert_tokens)
    
    if verbose:
        print(f'BERT tokens: {bert_tokens}')
        print(f'CoreNLP tokens: {corenlp_tokens}')
        print(f'CoreNLP dependency tree: {parse_tree_corenlp}')
        print(f'CoreNLP to BERT: {corenlp_to_bert_map}')
        
    output_bert_v1s = []
    output_bert_v2s = []
    types = []
    for edge in parse_tree_corenlp:
        (t, corenlp_v1, corenlp_v2) = edge
        
        if (corenlp_v1 not in corenlp_to_bert_map) or (corenlp_v2 not in corenlp_to_bert_map):
            continue;
        
        bert_v1 = corenlp_to_bert_map[corenlp_v1]
        bert_v2 = corenlp_to_bert_map[corenlp_v2]

        if (len(bert_v1) > 0) and (len(bert_v2) > 0):
            bert_v1_super = bert_v1[0]
            bert_v2_super = bert_v2[0]
            
            output_bert_v1s.append(bert_v1_super)
            output_bert_v2s.append(bert_v2_super)
            types.append(t)
            
            for bert_v1_sub in bert_v1[1:]:
                if (bert_v1_super, bert_v1_sub, "sprwrd") not in check_existed_dict:
                    output_bert_v1s.append(bert_v1_super)
                    output_bert_v2s.append(bert_v1_sub)
                    types.append("sprwrd")
                    check_existed_dict[(bert_v1_super, bert_v1_sub, "sprwrd")] = True
                
            for bert_v2_sub in bert_v2[1:]:
                if (bert_v2_super, bert_v2_sub, "sprwrd") not in check_existed_dict:
                    output_bert_v1s.append(bert_v2_super)
                    output_bert_v2s.append(bert_v2_sub)
                    types.append("sprwrd")
                    check_existed_dict[(bert_v2_super, bert_v2_sub, "sprwrd")] = True
                
    return output_bert_v1s, output_bert_v2s, types

In [9]:
def parse_data_sample(objs):
    output = []
    for sentence in tqdm(objs, total=len(objs)):
        aspects = sentence['aspects']
        for aspect in aspects:
            source_dep, target_dep, edge_type = build_dep_parse_tree(sentence['text'])
            output.append({
                'id': sentence['id'],
                'text': sentence['text'],
                'term': aspect['term'],
                'from': aspect['from'],
                'to': aspect['to'],
                'source_dep': source_dep,
                'target_dep': target_dep,
                'edge_type': edge_type,
                'label': aspect['polarity'],
            })
    return output

In [ ]:
output_path01 = '/home/hoang/github/BERT_ABSA/dataset/preprocessed_data/Laptops_Test.pkl'
objs = pd.read_pickle(output_path01)
for s in tqdm(objs, total=len(objs)):
#     tokenize_and_depparse(s['text'])
    build_dep_parse_tree(s['text'])

  0%|          | 0/411 [00:00<?, ?it/s]

In [ ]:
dataset_files = {
    'restaurant': {
        'train': 'Restaurants_Train.xml',
        'test': 'Restaurants_Test.xml',
        'trial': 'Restaurants_Trial.xml'
    },
    'laptop': {
        'train': 'Laptops_Train.xml',
        'test': 'Laptops_Test.xml',
        'trial': 'Laptops_Trial.xml'
    }
}

In [6]:
for dsname, fnames in dataset_files.items():
    for g, fname in fnames.items():
        input_path = str(PROJ_PATH/ 'dataset/raw_data' / fname)
        output_path01 = str(PROJ_PATH/ 'dataset/preprocessed_data' / fname.replace('.xml', '.pkl'))
        output_path02 = str(PROJ_PATH/ 'dataset/preprocessed_data' / fname.replace('.xml', '_data.pkl'))
        print(f'Load: {input_path}')
        print(f'Save parsed XML to: {output_path01}')
        print(f'Save data to: {output_path02}\n')
        objs = parseXML(input_path)
        output = parse_data_sample(objs)
        pd.to_pickle(objs, output_path01)
        pd.to_pickle(output, output_path02)

Load: /home/hoang/github/BERT_ABSA/dataset/raw_data/Restaurants_Train.xml
Save: /home/hoang/github/BERT_ABSA/dataset/preprocessed_data/Restaurants_Train.pkl

Load: /home/hoang/github/BERT_ABSA/dataset/raw_data/Restaurants_Test.xml
Save: /home/hoang/github/BERT_ABSA/dataset/preprocessed_data/Restaurants_Test.pkl

Load: /home/hoang/github/BERT_ABSA/dataset/raw_data/Restaurants_Trial.xml
Save: /home/hoang/github/BERT_ABSA/dataset/preprocessed_data/Restaurants_Trial.pkl

Load: /home/hoang/github/BERT_ABSA/dataset/raw_data/Laptops_Train.xml
Save: /home/hoang/github/BERT_ABSA/dataset/preprocessed_data/Laptops_Train.pkl

Load: /home/hoang/github/BERT_ABSA/dataset/raw_data/Laptops_Test.xml
Save: /home/hoang/github/BERT_ABSA/dataset/preprocessed_data/Laptops_Test.pkl

Load: /home/hoang/github/BERT_ABSA/dataset/raw_data/Laptops_Trial.xml
Save: /home/hoang/github/BERT_ABSA/dataset/preprocessed_data/Laptops_Trial.pkl



## Dataset

In [2]:
data = list(pd.read_csv('../dataset/preprocessed_data/Laptops_Train.csv').T.to_dict().values())

In [2]:
class Dataset(Dataset):
    def __init__(self, data_dir, transformation='QA_M', num_classes=3, bert_tokenizer=None, max_length=0, seed=0):
        random.seed(seed)
        assert transformation in ['QA_M', 'MLI_M', 'KW_M'], 'Invalid transformation method'
        assert num_classes in [2, 3], 'Invalid num_classes'
        
        self.transformation = transformation
        self.bert_tokenizer = bert_tokenizer
        self.max_length = max_length
        self.polarity_dict = {'positive': 0, 'negative': 1, 'neutral': 2}
        
        # load data
#         self.data = list(pd.read_csv(data_dir).T.to_dict().values())
        self.data = pd.read_csv(data_dir).to_dict('records')
        if num_classes == 2:
            self.data = [d for d in self.data if d['label'] != 'neutral']
    
    def transform(self, sample):
        seq1 = sample['text'].lower()
        term = sample['term'].lower()
        
        if self.transformation == 'QA_M':
            seq2 = f'what is the polarity of {term} ?'
            label = self.polarity_dict[sample['label']]
        elif self.transformation == 'MLI_M':
            seq2 = term.lower()
            label = self.polarity_dict[sample['label']]
        elif self.transformation == 'KW_M':
            seq2 = term
            label = self.polarity_dict[sample['label']]
        
        return seq1, seq2, label
        
    def encode_text(self, seq1, seq2):
        # encode
        encoded_text = self.bert_tokenizer.encode_plus(
            seq1,
            seq2,
            add_special_tokens=True,  # Add [CLS] and [SEP]
            max_length=self.max_length,  # maximum length of a sentence
            padding='max_length',  # Add [PAD]s
            truncation=True, # Truncate up to maximum length
            return_attention_mask=True,  # Generate the attention mask
            return_tensors='pt',  # Ask the function to return PyTorch tensors
        )
        return encoded_text
        
    def __getitem__(self, item):
        '''
        example = {
            'id': 1000,
            'text': 'The food is good, especially their more basic dishes, and the drinks are delicious.',
            'term': 'food',
            'label': 'positive',
            }
        '''
            
        # encoder
        sample = self.data[item]
        seq1, seq2, label = self.transform(sample)
        encoded_text = self.encode_text(seq1, seq2)

        single_input = {
            'id': sample['id'],
            'text': sample['text'],
            'seq1': seq1,
            'seq2': seq2,
            'term': sample['term'],
            'label': label, 
            'input_ids': encoded_text['input_ids'].flatten(),
            'token_type_ids': encoded_text['token_type_ids'].flatten(),
            'attention_mask': encoded_text['attention_mask'].flatten(),
        }
        return single_input

    def __len__(self):
        return len(self.data)
    
class DataModule(pl.LightningDataModule):
    def __init__(self, params):
        super().__init__()
        self.save_hyperparameters(params)

    def setup(self, stage=None):
        bert_tokenizer = BertTokenizer.from_pretrained(self.hparams.bert_name)
        
        # Assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:
            data_fit = Dataset(
                data_dir=self.hparams.data_train_dir,
                transformation=self.hparams.transformation,
                num_classes=self.hparams.num_classes,
                bert_tokenizer=bert_tokenizer,
                max_length=self.hparams.max_length,
                seed=self.hparams.seed)
            
            total_samples = data_fit.__len__()
            train_samples = int(data_fit.__len__() * 0.8)
            val_samples = total_samples - train_samples
            self.data_train, self.data_val = random_split(
                data_fit, [train_samples, val_samples], generator=torch.Generator().manual_seed(self.hparams.seed))

        # Assign test dataset for use in dataloader(s)
        if stage == "test" or stage is None:
            self.data_test = Dataset(
                data_dir=self.hparams.data_test_dir,
                transformation=self.hparams.transformation,
                num_classes=self.hparams.num_classes,
                bert_tokenizer=bert_tokenizer,
                max_length=self.hparams.max_length,
                seed=self.hparams.seed)

    def train_dataloader(self):
        return DataLoader(
            self.data_train, 
            batch_size=self.hparams.batch_size, 
            num_workers=4, 
            shuffle=False, # Already shuffle in random_split() 
            drop_last=True, 
#             collate_fn=lambda x: x,
        )
    
    def mytrain_dataloader(self):
        return DataLoader(
            self.data_train, 
            batch_size=self.hparams.batch_size, 
            num_workers=4, 
            shuffle=False, # Already shuffle in random_split() 
            drop_last=False, 
#             collate_fn=lambda x: x,
        )
    
    def val_dataloader(self):
        return DataLoader(
            self.data_val, 
            batch_size=self.hparams.batch_size, 
            num_workers=4, 
            shuffle=False,
#             drop_last=True, 
#             collate_fn=lambda x: x,
        )
    def test_dataloader(self):
        return DataLoader(
            self.data_test, 
            batch_size=self.hparams.batch_size, 
            num_workers=4, 
            shuffle=False,
#             drop_last=True, 
#             collate_fn=lambda x: x,
        )

## Model

In [6]:
class SentimentClassifier(pl.LightningModule):
    def __init__(self, params):
        super().__init__()
        self.save_hyperparameters(params)
        self.tokenizer = BertTokenizer.from_pretrained(self.hparams.pretrained_bert_name)
        self.bert = BertForSequenceClassification.from_pretrained(
            self.hparams.pretrained_bert_name, num_labels=3, output_hidden_states=True, output_attentions=True, return_dict=False)
        self.hidden_size = self.bert.config.hidden_size
        self.cross_entropy_loss = nn.CrossEntropyLoss()
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr)
        return optimizer  
    
    def forward(self, input_ids, attention_mask, token_type_ids, labels):
        loss, logits, hidden, _ = self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids,
            labels=labels,
        )
        
        return logits
    
    def margin_loss(self,  embedding_query, embedding_pos, embedding_neg):
        scores_pos = (embeddings_query * embeddings_pos).sum(dim=-1)
        scores_neg = (embeddings_query * embeddings_neg).sum(dim=-1) * self.scale
        return scores_pos - scores_neg
        
    def training_step(self, batch, batch_idx):
        # ['seq1', 'seq2', 'term', 'label', 'input_ids', 'token_type_ids', 'attention_mask']
        logits = self.forward(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            token_type_ids=batch['token_type_ids'],
            labels=batch['label'],
        )
        
        labels = batch['label']
        ce_loss = self.cross_entropy_loss(logits, labels)        
#         acc = utils.calc_accuracy(logits, labels).squeeze()
#         logs = {
#             'loss': ce_loss,
#             'acc': acc,
#         }
#         self.log_dict(logs, prog_bar=True)
        return ce_loss
    
    def validation_step(self, batch, batch_idx):
        logits = self.forward(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            token_type_ids=batch['token_type_ids'],
            labels=batch['label'],
        )
        
        labels = batch['label']
        ce_loss = self.cross_entropy_loss(logits, labels)        
        acc = utils.calc_accuracy(logits, labels).squeeze()
        macro_f1 = utils.calc_f1(logits, labels, avg_type='macro').squeeze()
        micro_f1 = utils.calc_f1(logits, labels, avg_type='micro').squeeze()

        logs = {
            'loss': ce_loss, 
            'acc': acc,
            'macro_f1': macro_f1,
            'micro_f1': micro_f1
        }
        self.log_dict(logs, prog_bar=True)
        return logs
    
    def validation_epoch_end(self, val_step_outputs):
        avg_loss = torch.stack([x['loss'] for x in val_step_outputs]).mean().cpu()
        avg_acc = torch.stack([x['acc'] for x in val_step_outputs]).mean().cpu()
        avg_macro_f1 = torch.stack([x['macro_f1'] for x in val_step_outputs]).mean().cpu()
        avg_micro_f1 = torch.stack([x['micro_f1'] for x in val_step_outputs]).mean().cpu()
        logs = {
            'val_loss': avg_loss, 
            'val_acc': avg_acc,
            'val_macro_f1': avg_macro_f1,
            'val_micro_f1': avg_micro_f1,
        }
        self.log_dict(logs, prog_bar=True)
     
    def test_step(self, batch, batch_idx):
        logits = self.forward(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            token_type_ids=batch['token_type_ids'],
            labels=batch['label'],
        )
        
        labels = batch['label']
        ce_loss = self.cross_entropy_loss(logits, labels)        
        acc = utils.calc_accuracy(logits, labels).squeeze()
        macro_f1 = utils.calc_f1(logits, labels, avg_type='macro').squeeze()
        micro_f1 = utils.calc_f1(logits, labels, avg_type='micro').squeeze()

        logs = {
            'loss': ce_loss, 
            'acc': acc,
            'macro_f1': macro_f1,
            'micro_f1': micro_f1
        }
        return logs
    
    def test_epoch_end(self, test_step_outputs):
        avg_loss = torch.stack([x['loss'] for x in test_step_outputs]).mean().cpu()
        avg_acc = torch.stack([x['acc'] for x in test_step_outputs]).mean().cpu()
        avg_macro_f1 = torch.stack([x['macro_f1'] for x in test_step_outputs]).mean().cpu()
        avg_micro_f1 = torch.stack([x['micro_f1'] for x in test_step_outputs]).mean().cpu()

        logs = {
            'test_loss': avg_loss, 
            'test_acc': avg_acc,
            'test_macro_f1': avg_macro_f1,
            'test_micro_f1': avg_micro_f1,
        }
        self.log_dict(logs, prog_bar=True)
        return logs
     

In [7]:
import commentjson
from collections import OrderedDict

def read_json(fname):
    '''
    Read in the json file specified by 'fname'
    '''
    with open(fname, 'rt') as handle:
        return commentjson.load(handle, object_hook=OrderedDict)

def build_model(config):
    data_params, model_params = config['data_params'], config['model_params']
    data = DataModule(data_params)
    model = SentimentClassifier(model_params)
    return data, model

In [5]:
def build_trainer(config, phase=None):
    trainer_params = config['trainer_params']
    data_params = config['data_params']
    
    # callbacks
    checkpoint = ModelCheckpoint(
        dirpath=trainer_params['checkpoint_dir'], 
        filename='{epoch}-{val_loss:.4f}-{val_acc:.4f}-{val_macro_f1:.4f}-{val_micro_f1:.4f}',
        save_top_k=trainer_params['top_k'],
        verbose=True,
        monitor=trainer_params['metric'],
        mode=trainer_params['mode'],
    )
    early_stopping = EarlyStopping(
        monitor='val_loss', 
        min_delta=0.00, 
        patience=trainer_params['patience'],
        verbose=False,
        mode=trainer_params['mode'],
    )
    metrics = {'loss': 'val_loss', 'acc': 'val_acc', 'macro_f1': 'val_macro_f1', 'micro_f1': 'val_micro_f1'}
    tuner = TuneReportCallback(metrics, on='validation_end')
    
    if phase == 'tune':
        callbacks = [tuner]
    else:
        callbacks = [checkpoint, early_stopping]
    
    # trainer_kwargs
    trainer_kwargs = {
        'max_epochs': trainer_params['max_epochs'],
        'gpus': 1 if torch.cuda.is_available() else 0,
    #     "progress_bar_refresh_rate": p_refresh,
    #     'gradient_clip_val': hyperparameters['grad_clip'],
        'weights_summary': 'full',
        'deterministic': True,
        'callbacks': callbacks,
    }

    trainer = Trainer(**trainer_kwargs)
    return trainer, trainer_kwargs

In [24]:
def execute(config, phase='train'):
    assert phase in ['train', 'test', 'tune'], 'Invalid phase!'
    seed_everything(config['data_params']['seed'], workers=True)
    data, clf = build_model(config)
    trainer, trainer_kwargs = build_trainer(config, phase=phase)
    
    if phase == 'train':
        trainer.fit(clf, data)
    elif phase == 'test':
        checkpoint_dir = Path(config['trainer_params']['checkpoint_dir'])
        print(f'Load checkpoint from: {str(checkpoint_dir)}')
        paths = sorted(checkpoint_dir.glob('*.ckpt'))
        for p in paths:
            print(p)
            model_test = SentimentClassifier.load_from_checkpoint(p)
            result = trainer.test(model_test, datamodule=data)

In [36]:
# parser = argparse.ArgumentParser(description='Training.')

# parser.add_argument('-config_file', help='config file path', default='../src/restaurant_config.json', type=str)
# parser.add_argument('-f', '--fff', help='a dummy argument to fool ipython', default='1')
# args = parser.parse_args()

# args.config = read_json(args.config_file)
# seed_everything(args.config['data_params']['seed'], workers=True)
# data, clf = build_model(args.config)
# trainer, trainer_kwargs = build_trainer(args.config)
# trainer.fit(clf, data)

## Predict

In [12]:
!ls /home/hoang/github/BERT_ABSA/src

__init__.py  config	 helper.py	 main.py   parser.py  utils.py
__pycache__  dataset.py  lightning_logs  model.py  tuner.py


In [14]:
parser = argparse.ArgumentParser(description='Training.')

parser.add_argument('-config_file', help='config file path', default='../src/config/restaurant_config.json', type=str)
parser.add_argument('-f', '--fff', help='a dummy argument to fool ipython', default='1')
args = parser.parse_args()

args.config = read_json(args.config_file)
execute(args.config, phase='test')

Global seed set to 12345
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized fro

Starting testing...
Load checkpoint from: ../model/restaurants
../model/restaurants/epoch=0-val_loss=0.7201-val_acc=0.6715-val_macro_f1=0.4689-val_micro_f1=0.6715.ckpt


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.7265625,
 'test_loss': 0.6642863154411316,
 'test_macro_f1': 0.49440622329711914,
 'test_micro_f1': 0.7265625}
--------------------------------------------------------------------------------
../model/restaurants/epoch=2-val_loss=0.6676-val_acc=0.7673-val_macro_f1=0.6605-val_micro_f1=0.7673.ckpt


...

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.8035300374031067,
 'test_loss': 0.5749001502990723,
 'test_macro_f1': 0.6690912246704102,
 'test_micro_f1': 0.8035300374031067}
--------------------------------------------------------------------------------
../model/restaurants/epoch=8-val_loss=0.6323-val_acc=0.8034-val_macro_f1=0.7262-val_micro_f1=0.8034.ckpt


/home/hoang/miniconda2/envs/py38/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are in

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.8460648059844971,
 'test_loss': 0.5475252270698547,
 'test_macro_f1': 0.7520278096199036,
 'test_micro_f1': 0.8460648059844971}
--------------------------------------------------------------------------------


/home/hoang/miniconda2/envs/py38/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(


## Tuning

In [33]:
import tempfile
import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import MedianStoppingRule

In [36]:
config = {
    "data_params": {
        "data_train_dir": "/home/hoang/github/BERT_ABSA/dataset/preprocessed_data/Restaurants_Train.csv",
        "data_test_dir": "/home/hoang/github/BERT_ABSA/dataset/preprocessed_data/Restaurants_Test.csv",
        "transformation": tune.choice(['QA_M', 'QA_B', 'MLI_M', 'MLI_B']),
        "num_classes": 3,
        "batch_size": 128,
        "bert_name": "bert-base-uncased",
        "max_length": 128,
        "seed": 12345
    },
    
    "model_params": {
        "pretrained_bert_name": "bert-base-uncased",
        "hidden_size": tune.choice([128, 256]),
        "hidden_dropout_prob": 0.1,
        "lr": tune.loguniform(1e-4, 1e-1),
    },
    "trainer_params": {
        "checkpoint_dir": "../model/restaurants",
        "top_k": 3,
        "max_epochs": 100,
        "metric": "val_loss",
        "patience": 10,
        "mode": "min",
        "tune": True,
        "train": False,
        "test": False,
    }
}

experiment_name = 'laptop'
experiment_dir = str(PROJ_PATH / 'experiment' / experiment_name )

if not os.path.exists(experiment_dir): os.mkdir(experiment_dir)
    
analysis = tune.run(
    tune.with_parameters(execute, phase='tune'),
    config=config,
    resources_per_trial={'gpu': 1},
    local_dir=experiment_dir,
    stop={'training_iteration': 20},
    scheduler=MedianStoppingRule(metric='acc'),
    num_samples=2)


2021-11-21 15:13:32,179	WARNING trial_runner.py:246 -- The maximum number of pending trials has been automatically set to the number of available cluster CPUs, which is high (280 CPUs/pending trials). If you're running an experiment with a large number of trials, this could lead to scheduling overhead. In this case, consider setting the `TUNE_MAX_PENDING_TRIALS_PG` environment variable to the desired maximum number of concurrent trials.


Trial name,status,loc,model_params/hidden_size,model_params/lr
execute_7fbfb_00000,PENDING,,64,0.00376421
execute_7fbfb_00001,PENDING,,64,0.024002


(ImplicitFunc pid=2684473) PROJ_PATH=/home/hoang/github/BERT_ABSA
(ImplicitFunc pid=2684473) PROJ_PATH=/home/hoang/github/BERT_ABSA
(ImplicitFunc pid=2684473) PROJ_PATH=/home/hoang/github/BERT_ABSA
(ImplicitFunc pid=2684464) PROJ_PATH=/home/hoang/github/BERT_ABSA
(ImplicitFunc pid=2684464) PROJ_PATH=/home/hoang/github/BERT_ABSA
(ImplicitFunc pid=2684464) PROJ_PATH=/home/hoang/github/BERT_ABSA


(pid=2684464) Global seed set to 12345
(pid=2684473) Global seed set to 12345


Trial name,status,loc,model_params/hidden_size,model_params/lr
execute_7fbfb_00000,RUNNING,141.117.3.96:2684464,64,0.00376421
execute_7fbfb_00001,RUNNING,141.117.3.96:2684473,64,0.024002


...

Epoch 19:  50%|█████     | 14/28 [00:06<00:05,  2.42it/s, loss=0.974, v_num=0, acc=0.621, macro_f1=0.255, micro_f1=0.621, val_loss=0.985, val_acc=0.610, val_macro_f1=0.252, val_micro_f1=0.610]


Trial name,status,loc,model_params/hidden_size,model_params/lr,iter,total time (s),loss,acc,macro_f1
execute_7fbfb_00000,RUNNING,141.117.3.96:2684464,64,0.00376421,19,257.346,0.984831,0.610487,0.252275
execute_7fbfb_00001,TERMINATED,141.117.3.96:2684473,64,0.024002,20,265.056,1.06616,0.610487,0.252275


Epoch 19:  79%|███████▊  | 22/28 [00:09<00:02,  2.40it/s, loss=0.974, v_num=0, acc=0.621, macro_f1=0.255, micro_f1=0.621, val_loss=0.985, val_acc=0.610, val_macro_f1=0.252, val_micro_f1=0.610]
Validating: 0it [00:00, ?it/s]4) 
Validating:   0%|          | 0/6 [00:00<?, ?it/s]
(ImplicitFunc pid=2684464) 
Epoch 19:  86%|████████▌ | 24/28 [00:10<00:01,  2.46it/s, loss=0.974, v_num=0, acc=0.621, macro_f1=0.255, micro_f1=0.621, val_loss=0.985, val_acc=0.610, val_macro_f1=0.252, val_micro_f1=0.610]
(ImplicitFunc pid=2684464) 
Validating:  33%|███▎      | 2/6 [00:00<00:01,  3.13it/s]
(ImplicitFunc pid=2684464) 
Epoch 19:  93%|█████████▎| 26/28 [00:10<00:00,  2.58it/s, loss=0.974, v_num=0, acc=0.621, macro_f1=0.255, micro_f1=0.621, val_loss=0.985, val_acc=0.610, val_macro_f1=0.252, val_micro_f1=0.610]
(ImplicitFunc pid=2684464) 
Validating:  67%|██████▋   | 4/6 [00:00<00:00,  5.05it/s]
(ImplicitFunc pid=2684464) 
Epoch 19: 100%|██████████| 28/28 [00:10<00:00,  2.70it/s, loss=0.974, v_num=0, ac

Trial name,status,loc,model_params/hidden_size,model_params/lr,iter,total time (s),loss,acc,macro_f1
execute_7fbfb_00000,TERMINATED,141.117.3.96:2684464,64,0.00376421,20,270.631,0.984872,0.610487,0.252275
execute_7fbfb_00001,TERMINATED,141.117.3.96:2684473,64,0.024002,20,265.056,1.06616,0.610487,0.252275


(pid=2684464) /home/hoang/miniconda2/envs/py38/lib/python3.8/site-packages/pytorch_lightning/trainer/deprecated_api.py:25: LightningDeprecationWarning: `Trainer.running_sanity_check` has been renamed to `Trainer.sanity_checking` and will be removed in v1.5.
(pid=2684464)   rank_zero_deprecation(
2021-11-21 15:18:05,920	INFO tune.py:630 -- Total run time: 273.75 seconds (273.62 seconds for the tuning loop).


In [ ]:
# tensorboard --logdir=./execute_2021-11-21_14-34-48 --port 1234